## Setting up the environment

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [3]:
import keras
import keras_nlp

## Load Dataset

In [4]:
import pandas as pd
new_data = pd.read_json('/kaggle/input/nlp-197/nlp-197.json')
new_data

,id,type,question,context,answer,cot_answer
0,seed_task_0,general,What is Spark NLP?,{'sentences': [['Learning with pseudo-ensemble...,Spark NLP is a Natural Language Processing (NL...,NaN
1,seed_task_1,general,What are the main features of Spark NLP?,{'sentences': [['vectors while the right side ...,Spark NLP comes with 1100+ pretrained pipeline...,NaN
2,seed_task_2,general,What is the impact of Spark NLP on healthcare?,"{'sentences': [['a sentence, the juxtaposition...",Spark NLP is used by 54% of healthcare organiz...,NaN
3,seed_task_3,general,What are the main challenges associated with S...,{'sentences': [['P(w i|cj)=P(cj|wi)P(wi)/summa...,There are no challenges associated with Spark ...,NaN
4,seed_task_4,general,What are the future prospects of Spark NLP?,{'sentences': [['whose performance has been co...,The text does not provide information about th...,NaN
...,...,...,...,...,...,...
875,seed_task_875,general,What is the most frequent language generation ...,{'sentences': [['Generation Rules Generation r...,Sequence-to-sequence Transformer architectures,NaN
876,seed_task_876,general,What are the two main types of summaries discu...,{'sentences': [['queries on knowledge graphs. ...,Extractive and abstractive summaries,NaN
877,seed_task_877,general,What is the Financial Narrative Summarization ...,{'sentences': [['the feasibility and signiﬁcan...,A shared task series for generating summaries ...,NaN
878,seed_task_878,general,What was the focus of the FNS-2023 workshop?,"{'sentences': [['34, pages 13001–13008. Chen Z...",Generating either extractive or abstractive su...,NaN


In [5]:
new_data['question'][0]

'What is Spark NLP?'

In [6]:
sentence = str(new_data['context'].iloc[0])
sentence = sentence.replace("{", "", 1)  # Removes only the first occurrence

In [7]:
for i, sentence in enumerate(new_data['context']):
    sentence = str(new_data['context'].iloc[i])
    sentence = sentence.replace("{", "").replace("}", "")
    new_data.at[i, 'context'] = sentence

In [8]:
new_data

,id,type,question,context,answer,cot_answer
0,seed_task_0,general,What is Spark NLP?,'sentences': [['Learning with pseudo-ensembles...,Spark NLP is a Natural Language Processing (NL...,NaN
1,seed_task_1,general,What are the main features of Spark NLP?,'sentences': [['vectors while the right side o...,Spark NLP comes with 1100+ pretrained pipeline...,NaN
2,seed_task_2,general,What is the impact of Spark NLP on healthcare?,"'sentences': [['a sentence, the juxtaposition ...",Spark NLP is used by 54% of healthcare organiz...,NaN
3,seed_task_3,general,What are the main challenges associated with S...,'sentences': [['P(w i|cj)=P(cj|wi)P(wi)/summat...,There are no challenges associated with Spark ...,NaN
4,seed_task_4,general,What are the future prospects of Spark NLP?,'sentences': [['whose performance has been con...,The text does not provide information about th...,NaN
...,...,...,...,...,...,...
875,seed_task_875,general,What is the most frequent language generation ...,'sentences': [['Generation Rules Generation ru...,Sequence-to-sequence Transformer architectures,NaN
876,seed_task_876,general,What are the two main types of summaries discu...,'sentences': [['queries on knowledge graphs. I...,Extractive and abstractive summaries,NaN
877,seed_task_877,general,What is the Financial Narrative Summarization ...,'sentences': [['the feasibility and signiﬁcant...,A shared task series for generating summaries ...,NaN
878,seed_task_878,general,What was the focus of the FNS-2023 workshop?,"'sentences': [['34, pages 13001–13008. Chen Zh...",Generating either extractive or abstractive su...,NaN


## Train Test Split

In [9]:
train_data = new_data[:440]
test_data = new_data[440:]

test_data

,id,type,question,context,answer,cot_answer
440,seed_task_440,general,What is the main topic of the text?,"'sentences': [['x ∈ K , suppose TKA(x)◦∩range(...",The text does not explicitly state a main topi...,NaN
441,seed_task_441,general,What is the purpose of the quality assurance c...,'sentences': [['to better grasp both evidence ...,The purpose of the quality assurance check is ...,NaN
442,seed_task_442,general,How many bibliographic records were retrieved?,'sentences': [['a polynomial proximity matrix ...,"A total of 1,717 bibliographic records were re...",NaN
443,seed_task_443,general,What metadata is included in each record?,'sentences': [['holds clear promise for energe...,Each record includes metadata such as the titl...,NaN
444,seed_task_444,general,What is the purpose of the filtering strategy?,'sentences': [['2020. Dynamic memory induction...,The filtering strategy is used to reduce the n...,NaN
...,...,...,...,...,...,...
875,seed_task_875,general,What is the most frequent language generation ...,'sentences': [['Generation Rules Generation ru...,Sequence-to-sequence Transformer architectures,NaN
876,seed_task_876,general,What are the two main types of summaries discu...,'sentences': [['queries on knowledge graphs. I...,Extractive and abstractive summaries,NaN
877,seed_task_877,general,What is the Financial Narrative Summarization ...,'sentences': [['the feasibility and signiﬁcant...,A shared task series for generating summaries ...,NaN
878,seed_task_878,general,What was the focus of the FNS-2023 workshop?,"'sentences': [['34, pages 13001–13008. Chen Zh...",Generating either extractive or abstractive su...,NaN


## Creating inference Data

In [10]:
!pip install -qU langchain-openai

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

In [11]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 31.0.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
import json
file_path = '/kaggle/input/nlp-chunks/chunks.json'
with open(file_path, 'r') as f:
    chunks = json.load(f)

In [14]:
print(len(chunks))

3926


In [15]:
!pip install einops

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00


In [16]:
print(chunks[0])

Spark NLP: Natural Language Understanding at Scale Veysel Kocaman, David Talby John Snow Labs Inc. 16192 Coastal Highway Lewes, DE , USA 19958 veysel, david@johnsnowlabs.com Abstract Spark NLP is a Natural Language Processing (NLP) library built on top of Apache Spark ML. It provides simple, performant & accurate NLP annotations for machine learning pipelines that can scale easily in a distributed environment. Spark NLP comes with 1100+ pretrained pipelines and models in more than 192+ languages. It supports nearly all the NLP tasks and modules that can be used seam- lessly in a cluster. Downloaded more than 2.7 million times and experiencing 9x growth since January 2020, Spark NLP is used by 54% of healthcare organizations as the world’s most widely used NLP library in the enterprise. Keywords: spark, natural language processing, deep learning, tensorﬂow, cluster 1. Spark NLP Library Natural language processing (NLP) is a key component in many data 


In [17]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.7 MB/s eta 0:00:00 0:00:01


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity: tensor([[0.5472, 0.6330]])


In [18]:
passage_embedding = model.encode(chunks)

Batches:   0%|          | 0/123 [00:00<?, ?it/s]

In [19]:
import torch
query_embedding = model.encode('What is spark NLP')
scores = util.dot_score(query_embedding, passage_embedding)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
values, indices = torch.topk(scores, 5)
print(values)
print(len(indices[0]))

tensor([[0.7582, 0.7382, 0.7009, 0.6594, 0.6387]])
5


In [21]:
chunks[int(indices[0][4])]

'driver node. There is a CuDA version of each TensorFlow component to enable training models on GPU when available. The Spark NLP is written in Scala and provides open-source API’s in Python, Java, Scala, and R - so that users do not need to be aware of the underlying implementation details (TensorFlow, Spark, etc.) in order to use it. Since it has an active release cycle (released 26 new versions in 2019 and another 26 in 2020), the latest trends and research in NLP ﬁeld are embraced and implemented rapidly in a way that could scale well in a cluster setting to allow common NLP pipelines run orders of magnitude faster than what the inherent design limitations of legacy libraries allowed. Spark NLP library has two versions: Open source and enterprise. Open source version has all the features and components that could be expected from any NLP library, using the '

In [22]:
#In a for loop. Generate the top 5 contexts for each question and append it to the question as the prompt
#Then get the answer and do inference

In [23]:
test_data['question'].iloc[0]

'What is the main topic of the text?'

In [24]:
inference_data = []
for i in range(0, test_data.shape[0]):
    question  = test_data['question'].iloc[i]
    
    query_embedding = model.encode(question)
    scores = util.dot_score(query_embedding, passage_embedding)
    values, indices = torch.topk(scores, 5)
    context = ""
    for j in range(len(indices[0])):
        context = context + "\n" + chunks[int(indices[0][j])]
        
    answer = test_data['answer'].iloc[i]
    template = f"Instruction:\n{question}\n\nContext:\n{context}"
    inference_data.append(template)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
print(len(inference_data))

440


## Load Base Model

In [59]:
from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1
text2text_generator = pipeline(model="mrm8488/t5-base-finetuned-question-generation-ap", device = device)
text2text_generator = pipeline("text2text-generation")
text2text_generator("question: What is 42 ? context: 42 is the answer to life, the universe and everything")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The `xla_device` argument

[{'generated_text': 'the answer to life, the universe and everything'}]

## Inference before fine tuning

In [58]:
print(inference_data[50])

Instruction:
What is the main architecture used by GreekBERT?

Context:

and NLG tasks. GreekBART (Evdaimon et al. 2024), based on the BART architecture (Lewis et al. 2020a), was pre-trained on the same datasets as GreekBERT plus the Greek Web Corpus (Outsios et al. 2018), incorporating diverse Greek text types, as well as formal and informal text, to enhance robustness. The GreekT5 series of models (Giarelis, Mastrokostas, and Karacapilidis 2024) were fine-tuned on the GreekSUM training dataset (Evdaimon et al. 2024), using the multilingual T5 LMs, which comprise (google/mt5-small (Xue et al. 2021), google/umt5-small (Chung et al. 2024a), and google/umt5-base (Chung et al. 2024a)). 19https://dumps.wikimedia.org/elwiki/ 20https://commoncrawl.org/ 13 Table 5 Monolingual Greek PLMs, including their availability (Yes: publicly available, Lmt: limited availability; tags are linkable; see Table 3 for details) and the backbone model they are base on. AUTHORS AVAILABILITY BACKBONE Giarelis, M

In [60]:
instruction = "What is the architecture of GreekBERT?"
context = """GreekBERT uses the BERT-BASE-UNCASED architecture (Devlin et al. 2019) 
and was pre-trained on 29 GB of Greek text from the Greek Wikipedia, the Greek part 
of the European Parliament Proceedings Parallel Corpus (Europarl), and the Greek part 
of OSCAR, a clean version of Common Crawl."""
answer = text2text_generator(f"question:{instruction} context: {context}")
print(answer)

[{'generated_text': 'BERT-BASE-UNCASED'}]


In [61]:
print(inference_data[1])
answer = text2text_generator(inference_data[1])
print(answer)

Instruction:
What is the purpose of the quality assurance check?

Context:

as it goes through continuous improvement cycles ( I1,I2,I3,I7). The goal is to quantify the occurrence of fairness issues ( I3) which may be valuable information for an audit. With every new model or change in data or parameters, the fairness dimension of the system may be affected ( I2). Interviewee one mentions the importance of standards that can be systematically checked with every system retraining ( I1). Such testing could be run at a very granular level assessing, for instance, confidence and explanation scores on a sentence level as a foundation for later judging fairness ( I2). Passing scores or red flags should be established for fairness test sets to make the fairness testing impactful ( I2,I3). Interviewees are divided on how automated such testing can be. While interviewees one and nine see potential in automated fairness testing, interviewee seven argues the process is not automatable as there is

In [62]:
for i in range(len(inference_data[:10])):
    print("Question: ", test_data['question'].iloc[i])
    answer = text2text_generator(inference_data[i])
    print("model answer: ", answer[0]['generated_text'])
    target_answer = test_data['answer'].iloc[i]
    print("Target answer: ", target_answer)
    print(end="\n")

Question:  What is the main topic of the text?
model answer:  a saturday night in california was a day of
Target answer:  The text does not explicitly state a main topic, therefore I cannot answer this question.

Question:  What is the purpose of the quality assurance check?
model answer:  False positives and negatives are important factors for evaluating fairness . 
Target answer:  The purpose of the quality assurance check is to ensure the accuracy and completeness of the retrieved records.

Question:  How many bibliographic records were retrieved?
model answer:  a total of 1,717 bibliographic records were retrieved, which were reduced to 1,
Target answer:  A total of 1,717 bibliographic records were retrieved.

Question:  What metadata is included in each record?
model answer:  aCL Anthology (AA) contains 209m papers and 2.5b citation
Target answer:  Each record includes metadata such as the title, author names, abstract, and publication date.

Question:  What is the purpose of the 

In [51]:
!pip install evaluate
!pip install bert_score
from evaluate import load
bertscore = load("bertscore")

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00


In [63]:
predictions = ["a saturday night in california was a day of", "general kenobi"]
references = ["hello world", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print(results)

{'precision': [0.595603346824646, 1.0], 'recall': [0.6710658073425293, 1.0], 'f1': [0.6310867071151733, 1.0], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.45.1)'}


In [64]:
predictions = []
references = []

for i in range(len(inference_data)):
    print("inference example: ", i)
    answer = text2text_generator(inference_data[i])
    predictions.append(answer[0]['generated_text'])
    
    target_answer = test_data['answer'].iloc[i]
    references.append(target_answer)

inference example:  0
inference example:  1
inference example:  2
inference example:  3
inference example:  4
inference example:  5
inference example:  6
inference example:  7
inference example:  8
inference example:  9
inference example:  10
inference example:  11
inference example:  12
inference example:  13
inference example:  14
inference example:  15
inference example:  16
inference example:  17
inference example:  18
inference example:  19
inference example:  20
inference example:  21
inference example:  22
inference example:  23
inference example:  24
inference example:  25
inference example:  26
inference example:  27
inference example:  28
inference example:  29
inference example:  30
inference example:  31
inference example:  32
inference example:  33
inference example:  34
inference example:  35
inference example:  36
inference example:  37
inference example:  38
inference example:  39
inference example:  40
inference example:  41
inference example:  42
inference example:  4

In [65]:
print(len(predictions))
print(len(references))

440
440


In [66]:
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
score = sum(results['f1']) / len(results['f1'])
print(score)

0.6915922276675701


## Fine-tune on train data

In [26]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [27]:
train_data.head()

,id,type,question,context,answer,cot_answer
0,seed_task_0,general,What is Spark NLP?,'sentences': [['Learning with pseudo-ensembles...,Spark NLP is a Natural Language Processing (NL...,NaN
1,seed_task_1,general,What are the main features of Spark NLP?,'sentences': [['vectors while the right side o...,Spark NLP comes with 1100+ pretrained pipeline...,NaN
2,seed_task_2,general,What is the impact of Spark NLP on healthcare?,"'sentences': [['a sentence, the juxtaposition ...",Spark NLP is used by 54% of healthcare organiz...,NaN
3,seed_task_3,general,What are the main challenges associated with S...,'sentences': [['P(w i|cj)=P(cj|wi)P(wi)/summat...,There are no challenges associated with Spark ...,NaN
4,seed_task_4,general,What are the future prospects of Spark NLP?,'sentences': [['whose performance has been con...,The text does not provide information about th...,NaN


In [28]:
!pip install --upgrade transformers datasets


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 61.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1


In [29]:
print(train_data.dtypes)
print(train_data.iloc[0])

id             object
type           object
question       object
context        object
answer         object
cot_answer    float64
dtype: object
id                                                  seed_task_0
type                                                    general
question                                     What is Spark NLP?
context       'sentences': [['Learning with pseudo-ensembles...
answer        Spark NLP is a Natural Language Processing (NL...
cot_answer                                                  NaN
Name: 0, dtype: object


In [30]:
print(train_data['answer'][2])

Spark NLP is used by 54% of healthcare organizations as the world’s most widely used NLP library in the enterprise.


In [31]:
# Convert list entries in 'answer' column to strings
train_data['answer'] = train_data['answer'].apply(
    lambda x: ' '.join(x) if isinstance(x, list) else x
)

/tmp/ipykernel_30/2757648839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['answer'] = train_data['answer'].apply(


In [32]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)

In [33]:
print(train_dataset['question'][1])
print(train_dataset['context'][1])
print(train_dataset['answer'][1])

What are the main features of Spark NLP?
'sentences': [['vectors while the right side of the equation is a scalar, to avoid Fobfuscating the linear structure, a dot product is used: 22 2 Word Representation F/parenleftbig (wi−wj)/latticetop˜wk/parenrightbig =Pik Pjk. (2.24) The model keeps the invariance under relabeling the target word and context word. It requires Fto be a homomorphism between the groups (R,+)and(R>0,×).The solution is F=exp. Then w/latticetop i˜wk=logNik−logNi. (2.25) To keep exchange symmetry, log Niis eliminated by adding biases biand˜bk.T h e model becomes w/latticetop i˜wk+bi+˜bk=logNik, (2.26) which is signiﬁcantly simpler than Eq. ( 2.21 ). The loss function is deﬁned as L=|V|/summationdisplay i,j=1f(Nij)(w/latticetop i˜wj+bi+˜bj−logNij), (2.27) where f(·)is a weighting function: f(x)=/braceleftBigg (x/xmax)αifx<xmax, 1 otherwise .(2.28) 2.4 Contextualized Word Representation In natural language, the meaning of an individual word usually relates to its context

In [34]:
def preprocess_function(examples):
    inputs = [
        f"question: {q} context: {c}" for q, c in zip(examples['question'], examples['context'])
    ]
    targets = examples['answer']
    model_inputs = tokenizer(inputs, max_length=1500, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/440 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4117: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [35]:
tokenized_dataset

Dataset({
    features: ['id', 'type', 'question', 'context', 'answer', 'cot_answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 440
})

In [38]:
import torch

torch.cuda.empty_cache()


In [40]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    eval_strategy="no",  # Disable evaluation
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Step,Training Loss
500,0.501600


TrainOutput(global_step=660, training_loss=0.4194406336004084, metrics={'train_runtime': 449.9302, 'train_samples_per_second': 2.934, 'train_steps_per_second': 1.467, 'total_flos': 2354954342400000.0, 'train_loss': 0.4194406336004084, 'epoch': 3.0})

In [41]:
trainer.save_model('./fine_tuned_t5_model')
tokenizer.save_pretrained('./fine_tuned_t5_model')

('./fine_tuned_t5_model/tokenizer_config.json',
 './fine_tuned_t5_model/special_tokens_map.json',
 './fine_tuned_t5_model/spiece.model',
 './fine_tuned_t5_model/added_tokens.json')

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.5 billion to 1.3 million).

## Load Fine-tuned model


In [42]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = '/kaggle/working/fine_tuned_t5_model'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [43]:
device = 0 if torch.cuda.is_available() else -1
text2text_generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device = device)

## Inference after fine-tuning
After fine-tuning, responses follow the instruction provided in the prompt.

In [44]:
print(inference_data[0])
answer = text2text_generator(inference_data[0])
print(answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (1324 > 512). Running this sequence through the model will result in indexing errors


Instruction:
What is the main topic of the text?

Context:

topic comprised of four broad topics: 1.Introduction: An overview of NLP and its use- fulness in Economics; Python fundamentals 2.Python & textual data, which focused on data collection, text extraction, pre-processing and text representation 3.NLP methods for economics, covering ex- ploratory corpus analysis, text classiﬁcation, topic modeling, and giving an overview of others such as information extraction and text summarization 4.NLP in Economics - research paper readings and discussion The students had to do 3 assignments covering the ﬁrst three topics which involved writing small python programs to use existing tools and write an analysis of how they work for their domain data. They also did a group presentation picking a paper that used NLP methods to address research ques- tions in economics, chosen most often from their own disciplinary journals, instead of NLP confer- ences. Students also had to submit a term paper wh

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'The text comprises four broad topics: Introduction to NLP and its use- fulness in Economics'}]


In [45]:
predictions = []
references = []

for i in range(len(inference_data)):
    print("inference example: ", i)
    answer = text2text_generator(inference_data[i])
    predictions.append(answer[0]['generated_text'])
    
    target_answer = test_data['answer'].iloc[i]
    references.append(target_answer)

inference example:  0
inference example:  1
inference example:  2
inference example:  3
inference example:  4
inference example:  5
inference example:  6
inference example:  7
inference example:  8


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


inference example:  9
inference example:  10
inference example:  11
inference example:  12
inference example:  13
inference example:  14
inference example:  15
inference example:  16
inference example:  17
inference example:  18
inference example:  19
inference example:  20
inference example:  21
inference example:  22
inference example:  23
inference example:  24
inference example:  25
inference example:  26
inference example:  27
inference example:  28
inference example:  29
inference example:  30
inference example:  31
inference example:  32
inference example:  33
inference example:  34
inference example:  35
inference example:  36
inference example:  37
inference example:  38
inference example:  39
inference example:  40
inference example:  41
inference example:  42
inference example:  43
inference example:  44
inference example:  45
inference example:  46
inference example:  47
inference example:  48
inference example:  49
inference example:  50
inference example:  51
inference ex

In [57]:
print(len(predictions))
print(predictions[10])
print(references[10])

440
The text is intended to provide a framework for understanding the structure of sentences and generating meaningful
To describe the process of surveying papers retrieved from databases


In [52]:
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
score = sum(results['f1']) / len(results['f1'])
print(score)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

0.7559248475865884
